### Part 1: 

This ended up taking me a long time and I did review a few others solutions, blog posts and eventually a helpful stackoverflow post. A good learning experience overall! In the end the following stack post was a solution I needed: https://stackoverflow.com/a/24471320/5094697

### Recursion Example: Almost Step by Step

My initial graph looks like below for first test case:

```python:

defaultdict(<class 'list'>, {'start': ['A', 'b'], 'A': ['start', 'c', 'b', 'end'], 'b': ['start', 'A', 'd', 'end'], 'c': ['A'], 'd': ['b'], 'end': ['A', 'b']})
```

Let's follow the path from `start` and walk through examples (traversing the `A` branch). `func` just means that we call a function (trying to highlight the recursive aspect). 

General pseudo-code:

```bash

- update path with current node
- check if at end:
    - if yes, return path
- loop over all possible nodes for current node (e.g. the values of the key node)
    - check if we have seen the node already (only matters for lowercase) 
        - call function with the new node and the updated path
        - once we have finished rcursion for the new node we loop over all new paths and append onto a path list (i just use a set)
- at the end we return the set of paths

```
Actually walking through steps: 

- func: set our initial path to `start`
    - confirm we aren't at node `end`
    - `start`: Iterate through `[A,B]`. Let's focus on `A`
        - since `A` hasn't been added to our path yet we recursively call our function with `A` as our node and our path set to `[start]`
    
- func: set our initial path to `[start,A]`, current node would be `A`
    - confirm we aren't at node `end`
    - `A`: Iterate through `[start, c,b, end]`.
    - since `start` has been added already and is lowercase we ignore, and move onto `c`
        - func: set our initial path to `[start,A, c]`, current node would be `c`
            - confirm we aren't at node `end`
            - `c`: Iterate through `[A]`.
            - since `A` is in our path but uppercase we recursively call function with `A` and path `[start,A, c]`
                - func: set our initial path to `[start,A, c, A]`, current node `A`
                - confirm we aren't at node `end`
                - `A`: Iterate through `[start, c,b, end]`, but `start` and `c` are off-limits now due to lower case rule.
                - basically this will run until we can output the following which show all possible paths for `start-A-c`
                    - {start,A, c, A, end}
                    - {start,A, c, A, b, end}
                    - {start,A, c, A, b, A, end}
                    
                 - For the above we hit end three times and return the single list, which is then added to a set once we get through all of the `c` options as: `[[start,A, c, A, end], [start,A, c, A, b, end], [start,A, c, A, b, A, end]]`

    - afer `c` is iterated over completely we move to `b`, which would give us the following: 
        - {'start,A,b,end', 'start,A,b,A,c,A,end', 'start,A,b,A,end'}

    - finally we go to `end`, which is a single path: `{start,A,end}`
    
- After this we are at 7 paths. We then move from `start -> b`, eventually giving us:
    - {start,b,A,c,A,end}
    - {start,b,A,end}
    - {start,b,end}
    
The ten total paths mentioned have been traversed. 


### Logic of Function: 

- `find_all_paths`:
    - recursive function that will accept a graph (I stored as a dictionary of possible steps), start node ('start'), end node ('end') and list for path (default is empty list)
    - I modified storage to a set and combined path into single string for more efficiencies (assume actual inputs get lots of paths)
        - Note: Since lists are mutable they aren't hashable, and python does not let us add lists to a set which is why I used the `.join()`
    
- Walking through steps:
    - Take the current path and add on start variable
        - Why does it need to be addition of [start] instead of append? 
            - This comes down to `path` default being `[]`
            - In python `path.append(start)` would just be treated as `[].append(start)` which is `None` (shown in debugger below)
    - if `start` == `end` then we have a path:
        - we should return this path as an element in a list. Why as an element in a list? 
            - This has to do with how returns occur in recursion
            - once a node is fully searched through it is going to return `paths`, which will be a list or a set of all paths found. 
            - We then need to iterate through that in the step that adds paths to our `paths` variable. 
            - If we return just the path then we run into an annoying situation on the step where we update `paths` and we would need to determine first if we had just a list versus having a list of lists (or set of string-paths)
    - I then create a `set()` called `paths` to store each path:
        - It helps me to think of this as being the `outer-most` set, so after all recursion is done and a `node` has no other nodes to traverse it will populate `newpaths`, which will then be added to `paths` set. 
    - iterate through all nodes of `start` variable (`graph[start]`):
        - what is happening?
            - at the beginning this is `A` and `b`
            - but when we get to `start == A` we see this becomes `['start', 'c', 'b', 'end']`, allowing for a variety of paths
        - we check if the node is in the path already OR is upper:
            - if it is upper then we don't care if in path and keep going
            - if it is not in the path then we keep doing
            - if it is lower and in path then we effectively move to the next neighbor - we have already visited this node in our current journey
        - assuming we have moved on:
            - we hit recursion -> we pass in our graph (unchanged), our node (which is our new starting point), our end (unchanged), and our path (which will have our journey so far). 
            - this will continue until we hit a return, which likely starts as returning a single path as an element in a list. 
        - our output is stored in newpaths, which we iterate through and add to paths
        - once all available nodes have been searched through we pass out our paths, and this happens until we have solved all paths from the initial `start`.

In [1]:
def find_all_paths_debugger(graph, start, end, path=[], debug = False):
        
        # debugging purposes: why this way instead of .append()?
        last_path = path.copy()
        path = path + [start]
        
        if start == end:
            if debug:
                print(f"Returning {[last_path.append(start)]} vs {[path]}")
            return [path]
        paths = set()
        for node in graph[start]:
            if node not in path or node.isupper():
                newpaths = find_all_paths_debugger(graph, node, end, path, debug)
                for newpath in newpaths:
                    paths.add(''.join(newpath))
                    if debug:
                        print(f"Searched node {node}, paths now {paths}")
        if debug:
            print(f"Finished searching from start {start}")
            print(f"Returning found paths: {paths}")
        return paths

In [2]:
from collections import defaultdict

# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12_testa.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
len(find_all_paths_debugger(fwd, 'start', 'end', [], True))

defaultdict(<class 'list'>, {'start': ['A', 'b'], 'A': ['start', 'c', 'b', 'end'], 'b': ['start', 'A', 'd', 'end'], 'c': ['A'], 'd': ['b'], 'end': ['A', 'b']})
Returning [None] vs [['start', 'A', 'c', 'A', 'b', 'A', 'end']]
Searched node end, paths now {'startAcAbAend'}
Finished searching from start A
Returning found paths: {'startAcAbAend'}
Searched node A, paths now {'startAcAbAend'}
Finished searching from start d
Returning found paths: set()
Returning [None] vs [['start', 'A', 'c', 'A', 'b', 'end']]
Searched node end, paths now {'startAcAbend', 'startAcAbAend'}
Finished searching from start b
Returning found paths: {'startAcAbend', 'startAcAbAend'}
Searched node b, paths now {'startAcAbend'}
Searched node b, paths now {'startAcAbend', 'startAcAbAend'}
Returning [None] vs [['start', 'A', 'c', 'A', 'end']]
Searched node end, paths now {'startAcAbend', 'startAcAend', 'startAcAbAend'}
Finished searching from start A
Returning found paths: {'startAcAbend', 'startAcAend', 'startAcAbAend'

10

In [3]:
### Without debugging: making list copies will get messy further on
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        
        if start == end:
            return [path]
        paths = set()
        for node in graph[start]:
            if node not in path or node.isupper():
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.add(''.join(newpath))
        return paths

In [4]:
# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12_testb.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
len(find_all_paths(fwd, 'start', 'end'))

defaultdict(<class 'list'>, {'dc': ['end', 'start', 'HN', 'LN', 'kj'], 'end': ['dc', 'HN'], 'HN': ['start', 'dc', 'end', 'kj'], 'start': ['HN', 'kj', 'dc'], 'kj': ['start', 'sa', 'HN', 'dc'], 'LN': ['dc'], 'sa': ['kj']})


19

In [5]:
# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12_testc.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
len(find_all_paths(fwd, 'start', 'end'))

defaultdict(<class 'list'>, {'fs': ['end', 'he', 'DX', 'pj'], 'end': ['fs', 'zg'], 'he': ['DX', 'fs', 'pj', 'RW', 'WI', 'zg'], 'DX': ['he', 'start', 'pj', 'fs'], 'start': ['DX', 'pj', 'RW'], 'pj': ['DX', 'zg', 'he', 'RW', 'start', 'fs'], 'zg': ['end', 'sl', 'pj', 'RW', 'he'], 'sl': ['zg'], 'RW': ['he', 'pj', 'zg', 'start'], 'WI': ['he']})


226

In [6]:
# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
len(find_all_paths(fwd, 'start', 'end'))

defaultdict(<class 'list'>, {'mj': ['TZ', 'LY', 'start', 'TH'], 'TZ': ['mj', 'ez', 'sb', 'start', 'uw'], 'start': ['LY', 'mj', 'TZ'], 'LY': ['start', 'uw', 'mj', 'end', 'ez'], 'TX': ['ez', 'sb', 'mt'], 'ez': ['TX', 'uw', 'TZ', 'TH', 'LY'], 'uw': ['ez', 'sb', 'LY', 'RR', 'vn', 'TZ'], 'TH': ['vn', 'end', 'mj', 'ez'], 'vn': ['TH', 'sb', 'uw'], 'sb': ['uw', 'TX', 'TZ', 'end', 'vn'], 'end': ['TH', 'LY', 'sb'], 'RR': ['uw'], 'mt': ['TX']})


3563

### Modifying for Part 2: 

- We can go through lowercase caves twice now....oh boy.

In [7]:
test = ['aa', 'aa','bb']
test.count('aa')

2

In [8]:
### Without debugging: making list copies will get messy further on
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        
        if start == end:
            return [path]
        paths = set()
        for node in graph[start]:
            # check if any lower case exceed 
            lw = [x for x in path if x.islower() and path.count(x) > 1]
            if len(lw) > 0:
                exclude_list = [x for x in path if x.islower()]
                exclude_list.append('start')
            else:
                exclude_list = ['start']
            if node not in exclude_list or node.isupper():
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.add(''.join(newpath))
        return paths

In [9]:
# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12_testa.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
assert(len(find_all_paths(fwd, 'start', 'end')) == 36)

defaultdict(<class 'list'>, {'start': ['A', 'b'], 'A': ['start', 'c', 'b', 'end'], 'b': ['start', 'A', 'd', 'end'], 'c': ['A'], 'd': ['b'], 'end': ['A', 'b']})


In [10]:
# Actual Inputs
# read data & store connections in default dict
fwd = defaultdict(list)
bwd = defaultdict(list)

with open('data/day12.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    for cxn in data:
        a,b = cxn.split('-')
        
        # add both directions
        fwd[a].append(b)
        fwd[b].append(a)

# review
print(fwd)

# run through function
len(find_all_paths(fwd, 'start', 'end'))

defaultdict(<class 'list'>, {'mj': ['TZ', 'LY', 'start', 'TH'], 'TZ': ['mj', 'ez', 'sb', 'start', 'uw'], 'start': ['LY', 'mj', 'TZ'], 'LY': ['start', 'uw', 'mj', 'end', 'ez'], 'TX': ['ez', 'sb', 'mt'], 'ez': ['TX', 'uw', 'TZ', 'TH', 'LY'], 'uw': ['ez', 'sb', 'LY', 'RR', 'vn', 'TZ'], 'TH': ['vn', 'end', 'mj', 'ez'], 'vn': ['TH', 'sb', 'uw'], 'sb': ['uw', 'TX', 'TZ', 'end', 'vn'], 'end': ['TH', 'LY', 'sb'], 'RR': ['uw'], 'mt': ['TX']})


105453